In [1]:
import astropy.io.fits as ap
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp2d
import numpy as np

import scipy
from scipy import integrate

import math

import pandas as pd

from joblib import Parallel, delayed
import multiprocessing
from mayavi import mlab


In [2]:
df=pd.read_csv("Coefficients_parallel.csv")
df = df.drop(columns=["Unnamed: 0" , 'index'])

In [3]:
N = 100
Ro = 69.634*1E9
Rss = 2.5 * Ro

In [4]:
X = np.linspace(-3*Ro ,3*Ro, N )
Y = np.linspace(-3*Ro ,3*Ro, N )
Z = np.linspace(-3*Ro ,3*Ro, N )


X , Y , Z = np.meshgrid(X,Y,Z)
X , Y , Z = X.flatten().T , Y.flatten().T , Z.flatten().T 

coords = np.vstack((X,Y,Z)).T

In [5]:
# s = mlab.points3d(X, Y, Z)
# mlab.show()

In [6]:
def appendSpherical_np(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    xy = xyz[:,0]**2 + xyz[:,1]**2
    ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
    ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
    #ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
    return ptsnew[:,3], ptsnew[:,4], ptsnew[:,5]

In [7]:
R , Theta , Phi = appendSpherical_np(coords)
Phi = (2*np.pi + Phi) * (Phi < 0) + Phi*(Phi > 0)

In [8]:
print(Phi.min(),Phi.max())
print(Theta.min(),Theta.max())

0.010100666585321924 6.273084640594265
0.014284013928896713 3.1273086396608964


In [9]:
spherical_coord = np.vstack((R,Theta,Phi)).T

In [10]:
condition_index = np.where((R>Ro) & (R<Rss))[0]

In [11]:
condition_index.shape

(275536,)

# Compute Magnetic Field B Radial

In [12]:
def Compute_BR(r, Phi, Theta , DF):
    Brad = 0
    
    for i in range(0,len(DF)):
        
        row = DF.iloc[i]
        
        l = row['l']
        m = row['m']
        
        glm = row['ans_r']
        hlm = row['ans_i']
        
        t1 = (Ro/r)**(l+2)
        t2 = (Ro/Rss)**(l+2)
        t3 = (r/Rss)**(l-1)
        
        t4 = (glm*np.cos(m*Phi)) + (hlm*np.sin(m*Phi))
        
        t5 = scipy.special.lpmv(m, l, np.cos(Theta))
    
        temp  =  ( (l+1)*t1 + l*(t2*t3) )
        
        Brad = Brad + (t5*t4*temp)
    

    return Brad

In [13]:
def Process_Br(k,Spherical_coordinates, Dframe):
    
 
    return Compute_BR(Spherical_coordinates[k,0],Spherical_coordinates[k,2],Spherical_coordinates[k,1],Dframe)
    

In [14]:
B_radial_condition = np.array(Parallel(n_jobs=4,verbose=10)(delayed(Process_Br)(i,spherical_coord,df) for i in condition_index))

B_radial = np.zeros(N**3)

B_radial[condition_index] = B_radial_condition

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1842s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1293s.) Setting batch_size=4.
[Parallel(n_jobs=4)]: Done  38 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1931s.) Setting batch_size=8.
[Parallel(n_jobs=4)]: Done  68 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 148 tasks      | elapsed:    2.7s
[Parallel(n_jobs=4)]: Done 236 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 340 tasks      | elapsed:    4.3s
[Parallel(n_jobs=4)]: Done 444 tasks      | elapsed:    5.1s
[Parallel(n_jobs=4)]: Done 564 tasks      | elapsed:    6.1s
[Paralle

# Compute Magnetic Field B Theta 

In [15]:
def Compute_BTheta(r, Phi, Theta , DF):
    
    h = np.pi*1E-3
    Bth = 0
    
    for i in range(0,len(DF)):
        
        row = DF.iloc[i]
        
        l = row['l']
        m = row['m']
        
        glm = row['ans_r']
        hlm = row['ans_i']
        
        t1 = (Ro/r)**(l+2)
        t2 = (Ro/Rss)**(l+2)
        t3 = (r/Rss)**(l-1)
        
        t4 = (glm*np.cos(m*Phi)) + (hlm*np.sin(m*Phi))
        
        t5 = ( scipy.special.lpmv(m, l, np.cos(Theta + h)) - scipy.special.lpmv(m, l, np.cos(Theta - h)) )/(2*h)
    
        temp  =  ( t1 - (t2*t3) )
        
        Bth = Bth  - (t5*t4*temp)
        
    return Bth

In [16]:
def Process_Btheta(k,Spherical_coordinates, Dframe):
    
 
    return Compute_BTheta(Spherical_coordinates[k,0],Spherical_coordinates[k,2],Spherical_coordinates[k,1],Dframe)

In [17]:
B_Theta_condition = np.array(Parallel(n_jobs=4,verbose=10)(delayed(Process_Btheta)(i,spherical_coord,df) for i in condition_index))

B_theta = np.zeros(N**3)

B_theta[condition_index] = B_Theta_condition

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Batch computation too fast (0.1043s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1220s.) Setting batch_size=4.
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1906s.) Setting batch_size=8.
[Parallel(n_jobs=4)]: Done  56 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 128 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 200 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 288 tasks      | elapsed:    2.7s
[Parallel(n_jobs=4)]: Done 376 tasks      | elapsed:    3.5s
[Parallel(n_jobs=4)]: Done 480 tasks      | elapsed:    4.5s
[Parallel(n_jobs=4)]: Done 584 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 704 tasks      | elapsed:    6.4s
[Paralle

# Compute Magnetic Field Component B Phi

In [18]:
def Compute_BPhi(r, Phi, Theta , DF):
    

    Bphi = 0
    
    for i in range(0,len(DF)):
        
        row = DF.iloc[i]
        
        l = row['l']
        m = row['m']
        
        glm = row['ans_r']
        hlm = row['ans_i']
        
        t1 = (Ro/r)**(l+2)
        t2 = (Ro/Rss)**(l+2)
        t3 = (r/Rss)**(l-1)
        
        t4 = (glm*np.sin(m*Phi)) - (hlm*np.cos(m*Phi))
        
        t5 = scipy.special.lpmv( m, l, np.cos(Theta) ) *(m/np.sin(Theta))
    
        temp  =  ( t1 - (t2*t3) )
        
        Bphi = Bphi  - (t5*t4*temp)
        
    return Bphi

In [19]:
def Process_Bphi(k,Spherical_coordinates, Dframe):
    
 
    return Compute_BPhi(Spherical_coordinates[k,0],Spherical_coordinates[k,2],Spherical_coordinates[k,1],Dframe)

In [20]:
B_Phi_condition = np.array(Parallel(n_jobs=4,verbose=10)(delayed(Process_Bphi)(i,spherical_coord,df) for i in condition_index))

B_phi = np.zeros(N**3)

B_phi[condition_index] = B_Phi_condition

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Batch computation too fast (0.0710s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1987s.) Setting batch_size=4.
[Parallel(n_jobs=4)]: Done  26 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  48 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done  84 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done 164 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 208 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done 260 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done 312 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 372 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 432 tasks      | elapsed:    4.7s
[Parallel(n_jobs=4)]: Done 50

# Accumalting All the Results

In [21]:
df_results = pd.DataFrame(columns=['X' , 'Y' , 'Z' , 'R' , 'Theta' , 'Phi' , 'B_Radial' , 'B_Theta' , 'B_Phi'])


In [22]:
df_results['X'] = X

df_results['Y'] = Y

df_results['Z'] = Z

df_results['R'] = R

df_results['Theta'] = Theta

df_results['Phi'] = Phi

df_results['B_Radial'] = B_radial

df_results['B_Theta'] = B_theta

df_results['B_Phi'] = B_phi

In [23]:
df_results

,X,Y,Z,R,Theta,Phi,B_Radial,B_Theta,B_Phi
0,-2.089020e+11,-2.089020e+11,-2.089020e+11,3.618289e+11,2.186276,3.926991,0.0,0.0,0.0
1,-2.089020e+11,-2.089020e+11,-2.046818e+11,3.594088e+11,2.176688,3.926991,0.0,0.0,0.0
2,-2.089020e+11,-2.089020e+11,-2.004615e+11,3.570223e+11,2.166972,3.926991,0.0,0.0,0.0
3,-2.089020e+11,-2.089020e+11,-1.962413e+11,3.546699e+11,2.157125,3.926991,0.0,0.0,0.0
4,-2.089020e+11,-2.089020e+11,-1.920210e+11,3.523523e+11,2.147148,3.926991,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
999995,2.089020e+11,2.089020e+11,1.920210e+11,3.523523e+11,0.994444,0.785398,0.0,0.0,0.0
999996,2.089020e+11,2.089020e+11,1.962413e+11,3.546699e+11,0.984467,0.785398,0.0,0.0,0.0
999997,2.089020e+11,2.089020e+11,2.004615e+11,3.570223e+11,0.974621,0.785398,0.0,0.0,0.0
999998,2.089020e+11,2.089020e+11,2.046818e+11,3.594088e+11,0.964904,0.785398,0.0,0.0,0.0


In [24]:
df_results.to_csv("Results_80.csv")